In [ ]:
import os
import numpy as np
import pydicom as dcm
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
## load in dicom image data
test_dcm = dcm.dcmread("test.dcm")

In [ ]:
test_dcm

In [ ]:
## load dicom image instance
test_img = test_dcm.pixel_array

In [ ]:
test_img.shape

In [ ]:
## view dicom image in Bone Window
plt.imshow(test_img, cmap=plt.cm.bone)

In [ ]:
## visualize dicom image instance in gray colormap
plt.imshow(test_img, cmap=plt.cm.gray)

In [ ]:
# Function to take care of teh translation and windowing. 
def window_image(img, window_center, window_width, intercept, slope, rescale=True):
    img = (img*slope +intercept) #for translation adjustments given in the dicom file. 
    img_min = window_center - window_width//2 #minimum HU level
    img_max = window_center + window_width//2 #maximum HU level
    img[img<img_min] = img_min #set img_min for all HU levels less than minimum HU level
    img[img>img_max] = img_max #set img_max for all HU levels higher than maximum HU level
    if rescale: 
        img = (img - img_min) / (img_max - img_min)*255.0 
    return img
    
def get_first_of_dicom_field_as_int(x):
    #get x[0] as in int is x is a 'pydicom.multival.MultiValue', otherwise get int(x)
    if type(x) == dcm.multival.MultiValue: return int(x[0])
    else: return int(x)
    
def get_windowing(data):
    dicom_fields = [data[('0028','1050')].value, #window center
                    data[('0028','1051')].value, #window width
                    data[('0028','1052')].value, #intercept
                    data[('0028','1053')].value] #slope
    return [get_first_of_dicom_field_as_int(x) for x in dicom_fields]

In [ ]:
window_center , window_width, intercept, slope = get_windowing(test_dcm)
window_center , window_width, intercept, slope

In [ ]:
test_win_img = window_image(test_img, window_center, window_width, intercept, slope, rescale=True)

In [ ]:
plt.imshow(test_win_img, cmap=plt.cm.gray)

In [ ]:
## manually adjust the window rage to visualize the image in bone window
window_center , window_width, intercept, slope = 300, 1500, -1024, 1
test_win_img = window_image(test_img, window_center, window_width, intercept, slope, rescale=True)
plt.imshow(test_win_img, cmap=plt.cm.gray)

In [ ]:
## directly converted PNG format image instance
Image.open("ori.png")